In [1]:
import os
import glob
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from utility.utility import *
from utility.algorithm import *

In [2]:
def import_files(path, dtype):
    '''
    Import files in the syntax that I used in the notebook. That is, choose the path that contains
    all the files, then obtain all dat and inf files as dictionaries that can be accessed by filename. 
    '''

    filename = "*.dat"
    files =  [ p for p in Path(path).glob(filename) ]
    file_dict = dict()
    for file in files:
        with open(file, 'rb') as f:
            # key of the dict is the filename, value the data.
            file_dict[os.path.basename(file)] = np.fromfile(f, np.float32)

    metadata_dict = dict()
    filename = "*.inf"
    files =  [ p for p in Path(path).glob(filename) ]
    for file in files:
        with open(file) as f:
            metadata_dict[os.path.basename(file)] = f.readlines()

    return file_dict, metadata_dict 

In [3]:
data_dict, metadata_dict = import_files(path="../../LOFAR_2e_jaars_RP_2022_voor_studenten/Data_Pulsar2", dtype="float32")
nbins_dict = dict()
dt_dict = dict()
mjd_dict = dict()
for key, val in metadata_dict.items():
    nbins_dict[key] = int(get_metadata_item(metadata_dict[key], 'Number of bins in the time series'))
    dt_dict[key] = float(get_metadata_item(metadata_dict[key], 'Width of each time series bin (sec)'))
    mjd_dict[key] = float(get_metadata_item(metadata_dict[key], 'Epoch of observation (MJD)'))

In [4]:
# use fitresults mathematica to make function 
x_range = [number for number in np.arange(0.20+ 5.63e4, 0.50 + 5.63e4, 0.001)]
B = 0.00166275
A = 7.66933e-8
omega = 39.2704
phi = 0.345537

#create y values
sine_fit = [B + A * np.sin(omega*number + phi) for number in x_range]

# errors from fit
B_err = 1.94857e-9
A_err = 2.62111e-9
omega_err = 1.05838
phi_err = 59587.

In [5]:
%matplotlib qt
# working data from mathematica that is from python
x = [56300.4688155539, 56300.45492639893, 56300.44103724313, 56300.42714808649, 56300.413258929024, 56300.39936977075, 56300.38548061164, 56300.37159145176, 56300.35770229113, 56300.34381312978]
y = [0.0016627957592090326, 0.00166282563896264, 0.0016628302415629193, 0.0016628131097909669, 0.0016627781654799023, 0.0016627323139534107, 0.0016626925155149316, 0.0016626750457100386, 0.0016626881480292916, 0.001662710837899844]
yerr = [2.353097663263867e-9, 2.3435228659575588e-9, 2.2291472249414805e-9, 1.9880488969239098e-9, 3.1889485827440945e-9, 3.46808484243367e-9, 2.1726783415015955e-9, 5e-9, 2.2821954279209845e-9, 3.615634992325844e-9]

list_MJD_All =  list( mjd_dict.values() )
# lines datapoints 
ymin = 8.4e-7 + 1.662e-3
ymax = 6.6e-7 + 1.662e-3
plt.clf()
plt.vlines(list_MJD_All, ymin, ymax, colors='yellow', linestyles='dotted')
plt.plot(x,y, 'o')
plt.plot(x_range, sine_fit, color='magenta')
plt.errorbar(x, y, yerr, color='red', fmt='o')

print(list_MJD_All)

[56299.61949397883, 56299.364628335425, 56300.44103724313, 56300.37159145176, 56300.38548061164, 56300.42714808649, 56300.34381312978, 56300.48270470806, 56300.4688155539, 56300.32992396778, 56300.413258929024, 56300.39936977075, 56300.35770229113, 56300.45492639893]


In [ ]:
%matplotlib qt
plt.clf()
plt.plot(x,y, 'o', color='red')
plt.plot(x_range, sine_fit)

{'L83084_SAP0_BEAM0_DM39.66.dat': array([41134. , 45705. , 44095. , ..., 44421.5, 44421.5, 44421.5],
      dtype=float32), 'L83079_SAP0_BEAM0_DM39.66.dat': array([43037. , 43082. , 41492. , ..., 44427.1, 44427.1, 44427.1],
      dtype=float32), 'L83083_SAP0_BEAM0_DM39.66.dat': array([43138.  , 43507.  , 44060.  , ..., 44436.46, 44436.46, 44436.46],
      dtype=float32), 'L83089_SAP0_BEAM0_DM39.66.dat': array([43402.   , 41204.   , 41362.   , ..., 44388.176, 44388.176,
       44388.176], dtype=float32), 'L83080_SAP0_BEAM0_DM39.66.dat': array([42803.   , 43653.   , 43815.   , ..., 44424.527, 44424.527,
       44424.527], dtype=float32), 'L83087_SAP0_BEAM0_DM39.66.dat': array([42469.   , 43798.   , 43904.   , ..., 44301.566, 44301.566,
       44301.566], dtype=float32), 'L83086_SAP0_BEAM0_DM39.66.dat': array([43764.   , 45682.   , 44446.   , ..., 44383.402, 44383.402,
       44383.402], dtype=float32), 'L83081_SAP0_BEAM0_DM39.66.dat': array([44707.  , 43506.  , 44092.  , ..., 44407.11, 44

In [12]:
from utility.utility import find_period_in_index, stacking

#period guess by using the intersection of the vertical line where the datapoint should've been and the sine fit

period_dataset3 = 16.828 # found with desmos
dt = 2.04799998755334e-05
nbins = 8800000
pulsar_data= data_dict['L83078_SAP0_BEAM0_DM39.66.dat']


period_in_index = find_period_in_index(period=period_dataset3, dt=dt)
stack = stacking(pulsar_data=pulsar_data, period_in_index=period_in_index, nbins=nbins)

In [13]:
time = [ dt * t for t in range(len(stack))]

plt.plot(time, stack / max( stack ) )#, color='magenta')
plt.xlabel('Time (s)')
plt.ylabel('Relative amplitude')

Text(0, 0.5, 'Relative amplitude')